In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = (
    SparkSession
    .builder
    .appName("MyApplication")
    .getOrCreate()
)

In [3]:
spark

In [4]:
data_path = "data/orders_copy.txt"
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [11]:
df.printSchema()
print('Initial DataFrame')
df.show()

root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- category: string (nullable = true)

Initial DataFrame
+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+



In [16]:
selected_columns = df.select("user_id", 'amount')
print('Selected Columns:')
selected_columns.show()

Selected Columns:
+-------+------+
|user_id|amount|
+-------+------+
|      1|  1500|
|      2|   800|
|      1|  2300|
|      3|  1200|
|      5|   900|
|      2|  3100|
|      4|   450|
+-------+------+



In [17]:
filtered_data = df.filter(
    df.user_id <= 3 
)
print('Filtered Data:')
filtered_data.show()

Filtered Data:
+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     106|      2|  3100|электроника|
+--------+-------+------+-----------+



In [19]:
filtered_data_2 = selected_columns.filter(
    selected_columns.user_id <= 3
)
print("Filtered Data on Selected Data:")
filtered_data_2.show()

Filtered Data on Selected Data:
+-------+------+
|user_id|amount|
+-------+------+
|      1|  1500|
|      2|   800|
|      1|  2300|
|      3|  1200|
|      2|  3100|
+-------+------+



In [36]:
group_data = df.groupBy("user_id").agg({"amount": "sum"})
group_data.show()

+-------+-----------+
|user_id|sum(amount)|
+-------+-----------+
|      1|       3800|
|      3|       1200|
|      5|        900|
|      4|        450|
|      2|       3900|
+-------+-----------+



In [35]:
df.show()

+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+



In [33]:
group_data2 = df.groupBy("category").agg({"amount": "sum", "user_id": "avg"})

In [34]:
group_data2.show()

+-----------+-----------+------------+
|   category|sum(amount)|avg(user_id)|
+-----------+-----------+------------+
|      книги|       1700|         3.5|
|электроника|       5800|         2.0|
|     одежда|       2750|         2.5|
+-----------+-----------+------------+



In [37]:
path_users = 'data/users.csv'
df2 = spark.read.csv(path_users, header=True, inferSchema=True)

In [43]:
print('Table Orders:')
df.show()
print('Table Users:')
df2.show()


Table Orders:
+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+

Table Users:
+-------+-----+---+------+
|user_id| name|age|  city|
+-------+-----+---+------+
|      1| Анна| 25|Москва|
|      2| Иван| 30|   СПБ|
|      3|Мария| 22|Москва|
|      4| Петр| 35|Казань|
|      5|Елена| 28|   СПБ|
+-------+-----+---+------+



In [46]:
df3 = df.join(
    df2,
    on="user_id",
    how='left'
)

In [47]:
df3.show()

+-------+--------+------+-----------+-----+---+------+
|user_id|order_id|amount|   category| name|age|  city|
+-------+--------+------+-----------+-----+---+------+
|      1|     101|  1500|электроника| Анна| 25|Москва|
|      2|     102|   800|      книги| Иван| 30|   СПБ|
|      1|     103|  2300|     одежда| Анна| 25|Москва|
|      3|     104|  1200|электроника|Мария| 22|Москва|
|      5|     105|   900|      книги|Елена| 28|   СПБ|
|      2|     106|  3100|электроника| Иван| 30|   СПБ|
|      4|     107|   450|     одежда| Петр| 35|Казань|
+-------+--------+------+-----------+-----+---+------+



In [48]:
df3.limit(1000).sort("user_id").show()

+-------+--------+------+-----------+-----+---+------+
|user_id|order_id|amount|   category| name|age|  city|
+-------+--------+------+-----------+-----+---+------+
|      1|     101|  1500|электроника| Анна| 25|Москва|
|      1|     103|  2300|     одежда| Анна| 25|Москва|
|      2|     102|   800|      книги| Иван| 30|   СПБ|
|      2|     106|  3100|электроника| Иван| 30|   СПБ|
|      3|     104|  1200|электроника|Мария| 22|Москва|
|      4|     107|   450|     одежда| Петр| 35|Казань|
|      5|     105|   900|      книги|Елена| 28|   СПБ|
+-------+--------+------+-----------+-----+---+------+



In [49]:
df4 = df.select("user_id", "category")

In [50]:
df4.show()

+-------+-----------+
|user_id|   category|
+-------+-----------+
|      1|электроника|
|      2|      книги|
|      1|     одежда|
|      3|электроника|
|      5|      книги|
|      2|электроника|
|      4|     одежда|
+-------+-----------+



In [51]:
df4 = (df.select("user_id", "category")
       .join(
           df2
           .select("user_id", "city"),
           on="user_id",
           how='left'
       )
)

In [52]:
df4.show()

+-------+-----------+------+
|user_id|   category|  city|
+-------+-----------+------+
|      1|электроника|Москва|
|      2|      книги|   СПБ|
|      1|     одежда|Москва|
|      3|электроника|Москва|
|      5|      книги|   СПБ|
|      2|электроника|   СПБ|
|      4|     одежда|Казань|
+-------+-----------+------+



In [54]:
df.show()

+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+



In [55]:
df2.show()

+-------+-----+---+------+
|user_id| name|age|  city|
+-------+-----+---+------+
|      1| Анна| 25|Москва|
|      2| Иван| 30|   СПБ|
|      3|Мария| 22|Москва|
|      4| Петр| 35|Казань|
|      5|Елена| 28|   СПБ|
+-------+-----+---+------+



In [56]:
df3.show()

+-------+--------+------+-----------+-----+---+------+
|user_id|order_id|amount|   category| name|age|  city|
+-------+--------+------+-----------+-----+---+------+
|      1|     101|  1500|электроника| Анна| 25|Москва|
|      2|     102|   800|      книги| Иван| 30|   СПБ|
|      1|     103|  2300|     одежда| Анна| 25|Москва|
|      3|     104|  1200|электроника|Мария| 22|Москва|
|      5|     105|   900|      книги|Елена| 28|   СПБ|
|      2|     106|  3100|электроника| Иван| 30|   СПБ|
|      4|     107|   450|     одежда| Петр| 35|Казань|
+-------+--------+------+-----------+-----+---+------+



In [59]:
sorted_data_df3 = df3.orderBy("user_id")

In [60]:
sorted_data_df3.show()

+-------+--------+------+-----------+-----+---+------+
|user_id|order_id|amount|   category| name|age|  city|
+-------+--------+------+-----------+-----+---+------+
|      1|     101|  1500|электроника| Анна| 25|Москва|
|      1|     103|  2300|     одежда| Анна| 25|Москва|
|      2|     102|   800|      книги| Иван| 30|   СПБ|
|      2|     106|  3100|электроника| Иван| 30|   СПБ|
|      3|     104|  1200|электроника|Мария| 22|Москва|
|      4|     107|   450|     одежда| Петр| 35|Казань|
|      5|     105|   900|      книги|Елена| 28|   СПБ|
+-------+--------+------+-----------+-----+---+------+



In [61]:
sort_df2 = (df2
            .orderBy(
                col("user_id").desc(),
                col("age").desc()
            )
)

In [62]:
sort_df2.show()

+-------+-----+---+------+
|user_id| name|age|  city|
+-------+-----+---+------+
|      5|Елена| 28|   СПБ|
|      4| Петр| 35|Казань|
|      3|Мария| 22|Москва|
|      2| Иван| 30|   СПБ|
|      1| Анна| 25|Москва|
+-------+-----+---+------+



In [65]:
test = df3.select("category").distinct()

In [66]:
test.show()

+-----------+
|   category|
+-----------+
|      книги|
|электроника|
|     одежда|
+-----------+



In [67]:
df.show()

+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+



In [71]:
dropped_df = df.drop("order_id", "amount")

In [72]:
dropped_df.show()

+-------+-----------+
|user_id|   category|
+-------+-----------+
|      1|электроника|
|      2|      книги|
|      1|     одежда|
|      3|электроника|
|      5|      книги|
|      2|электроника|
|      4|     одежда|
+-------+-----------+



In [73]:
df.show()

+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+



In [74]:
new_df = df.withColumn("new_column", df.amount * 2)
new_df.show()

+--------+-------+------+-----------+----------+
|order_id|user_id|amount|   category|new_column|
+--------+-------+------+-----------+----------+
|     101|      1|  1500|электроника|      3000|
|     102|      2|   800|      книги|      1600|
|     103|      1|  2300|     одежда|      4600|
|     104|      3|  1200|электроника|      2400|
|     105|      5|   900|      книги|      1800|
|     106|      2|  3100|электроника|      6200|
|     107|      4|   450|     одежда|       900|
+--------+-------+------+-----------+----------+



In [78]:
df_rename = df.withColumnRenamed("paw_paw", "new_column")
df_rename.show()

+--------+-------+------+-----------+
|order_id|user_id|amount|   category|
+--------+-------+------+-----------+
|     101|      1|  1500|электроника|
|     102|      2|   800|      книги|
|     103|      1|  2300|     одежда|
|     104|      3|  1200|электроника|
|     105|      5|   900|      книги|
|     106|      2|  3100|электроника|
|     107|      4|   450|     одежда|
+--------+-------+------+-----------+

